In [508]:
import pandas as pd
import random 
from collections import Counter

In [509]:
course_metadata = pd.read_csv(r"C:\My Progress\UE Internship\Version 02\Code\csv_dataframe - Sheet1.csv") 
# How many sperms are passes in one ejaculation? 
POPULATION_SIZE = 200

In [510]:
course_metadata

,Course ID,Section,Teacher ID,Credit Hours,Semester
0,CE413,A,Mr. Salman Saeed,3,8
1,CE413,B,Mr. Salman Saeed,3,8
2,CS439,A,Dr. Farhan Khan,3,8
3,CS440,B,Dr. Farhan Khan,3,8
4,CS441,C,Dr. Farhan Khan,3,8
...,...,...,...,...,...
97,CS367,D,Ms. Nazia Shehzadi,3,6
98,CS373,A,Dr. Fahad Bin Muslim,3,6
99,CS374,B,Dr. Fahad Bin Muslim,3,6
100,CS375,C,Dr. Fahad Bin Muslim,3,6


In [511]:
#How the sperms look like(Are they really greasy?)
GENES = list(range(0,len(course_metadata)))

In [512]:
# Create a dictionary where the keys are column names 'lh0' to 'lh14'
columns = [f'lh{i}' for i in range(10)]

# Create a DataFrame with 5 rows and 15 columns
data = [[f'Row{row+1}_Col{col}' for col in range(10)] for row in range(5)]

# Create the DataFrame
timetable = pd.DataFrame(data, columns=columns)

timetable

,lh0,lh1,lh2,lh3,lh4,lh5,lh6,lh7,lh8,lh9
0,Row1_Col0,Row1_Col1,Row1_Col2,Row1_Col3,Row1_Col4,Row1_Col5,Row1_Col6,Row1_Col7,Row1_Col8,Row1_Col9
1,Row2_Col0,Row2_Col1,Row2_Col2,Row2_Col3,Row2_Col4,Row2_Col5,Row2_Col6,Row2_Col7,Row2_Col8,Row2_Col9
2,Row3_Col0,Row3_Col1,Row3_Col2,Row3_Col3,Row3_Col4,Row3_Col5,Row3_Col6,Row3_Col7,Row3_Col8,Row3_Col9
3,Row4_Col0,Row4_Col1,Row4_Col2,Row4_Col3,Row4_Col4,Row4_Col5,Row4_Col6,Row4_Col7,Row4_Col8,Row4_Col9
4,Row5_Col0,Row5_Col1,Row5_Col2,Row5_Col3,Row5_Col4,Row5_Col5,Row5_Col6,Row5_Col7,Row5_Col8,Row5_Col9


In [513]:
temp_day_list = []

In [514]:
#It is a proper sex scene in which two people mate and generate a new being, the generated being fitness is also checked along side.
class Individual(object): 
	''' 
	Class representing individual in population 
	'''
	def __init__(self, chromosome): 
		self.chromosome = chromosome 
		self.fitness = self.cal_fitness() 

	@classmethod
	def mutated_genes(self): 
		''' 
		create random genes for mutation 
		'''
		global GENES 
		gene = random.choice(GENES) 
		return gene 

	@classmethod
	def create_gnome(self): 
		''' 
		create chromosome or string of genes 
		'''
		global TARGET
		# seven classes in a day
		gnome_len = 7 
		return [self.mutated_genes() for _ in range(gnome_len)] 

	def mate(self, par2): 
		''' 
		Perform mating and produce new offspring 
		'''

		# chromosome for offspring 
		child_chromosome = [] 
		for gp1, gp2 in zip(self.chromosome, par2.chromosome):	 

			# random probability 
			prob = random.random() 

			# if prob is less than 0.45, insert gene 
			# from parent 1 
			if prob < 0.45: 
				child_chromosome.append(gp1) 

			# if prob is between 0.45 and 0.90, insert 
			# gene from parent 2 
			elif prob < 0.90: 
				child_chromosome.append(gp2) 

			# otherwise insert random gene(mutate), 
			# for maintaining diversity 
			else: 
				child_chromosome.append(self.mutated_genes()) 

		# create new Individual(offspring) using 
		# generated chromosome for offspring 
		return Individual(child_chromosome)

	def cal_fitness(self):
         global TARGET 
         fitness = 0
         #credit hours validation
         for gs in zip(self.chromosome):
             if course_metadata.iloc[gs]['Credit Hours'] <=0:
                 fitness += 1
         #courses clash should be finished
         
         for i in range(0,len(self.chromosome)):
             #lh_tt means any perticular lecture hall timetable on any perticular day
             for lh_tt in temp_day_list:
                 if self.chromosome[i] == lh_tt[i]:
                     fitness += 1
                     
         # Teacher should not have clash with same class at same slot
         for i in range(0,len(self.chromosome)):
             #lh_tt means any perticular lecture hall timetable on any perticular day
             for lh_tt in temp_day_list:
                 if course_metadata.iloc[self.chromosome[i]]["Teacher ID"] == course_metadata.iloc[lh_tt[i]]["Teacher ID"]:
                     fitness += 1
         
         # Same section can not have two different courses classes at one time
         for i in range(0,len(self.chromosome)):
             #lh_tt means any perticular lecture hall timetable on any perticular day
             for lh_tt in temp_day_list:
                 if course_metadata.iloc[self.chromosome[i]]["Semester"] == course_metadata.iloc[lh_tt[i]]["Semester"]:
                     if course_metadata.iloc[self.chromosome[i]]["Section"] == course_metadata.iloc[lh_tt[i]]["Section"]:
                         fitness += 1
         
         # No course can occur more than more than two time in any list of timetable 
         
         occurances = Counter(self.chromosome)
         for key in occurances:
             if occurances[key] > 3:
                 fitness += 1
         for gs in zip(self.chromosome):
             if course_metadata.iloc[gs]['Credit Hours'] - occurances[key] <= 0:
                 fitness += 1
                    
         return fitness - 1 

In [515]:
timetable.iloc[0]['lh0']

'Row1_Col0'

In [516]:
def reduce_credit_hours(course_list):
    for course in course_list:
        course_metadata.loc[course, 'Credit Hours'] -= 1

In [517]:
# Driver code 
def main():
    global POPULATION_SIZE
    global temp_day_list
    ctr = 0 
    for i in range(0,5):
        print("Hello",i)
        temp_day_list = []
        for lecture_hall in timetable:
            #current generation 
            generation = 1
            found = False
            
            population = [] 
            for _ in range(POPULATION_SIZE):
                gnome = Individual.create_gnome() 
                population.append(Individual(gnome)) 
            while not found: 
                # sort the population in increasing order of fitness score 
                population = sorted(population, key = lambda x:x.fitness) 
				# if the individual having lowest fitness score ie. 
				# 0 then we know that we have reached to the target 
				# and break the loop 
                if population[0].fitness <= 0: 
                    found = True
                    timetable.loc[i,lecture_hall] = population[0].chromosome
                    # reduce the credit hours of each course in the offspring by 1
                    reduce_credit_hours(population[0].chromosome)
                    temp_day_list.append(population[0].chromosome)
                    ctr = ctr+1
                    print(ctr)
                    break
                if ctr >= 27:
                    print(len(population),"dddddddddddd")
                    print(population[0].chromosome, population[0].fitness)
                # Otherwise generate new offsprings for new generation 
                new_generation = [] 

				# Perform Elitism, that mean 10% of fittest population 
				# goes to the next generation 
                s = int((10*POPULATION_SIZE)/100) 
                new_generation.extend(population[:s]) 

				# From 50% of fittest population, Individuals 
				# will mate to produce offspring 
                s = int((80*POPULATION_SIZE)/100) 
                for _ in range(s): 
                    parent1 = random.choice(population[:50]) 
                    parent2 = random.choice(population[:50]) 
                    child = parent1.mate(parent2) 
                    new_generation.append(child)
                for _ in range(10):
                    gnome = Individual.create_gnome() 
                    new_generation.append(Individual(gnome)) 
                population = new_generation
                generation += 1

In [518]:
if __name__ == '__main__': 
	main() 

Hello 0
1
2
3
4
5
6
7
8
9
10
Hello 1
11
12
13
14
15
16
17
18
19
20
Hello 2
21
22
23
24
25
26
27
200 dddddddddddd
[76, 34, 7, 33, 32, 73, 97] 6
190 dddddddddddd
[64, 51, 73, 32, 29, 89, 33] 5
190 dddddddddddd
[35, 51, 9, 25, 32, 76, 50] 3
190 dddddddddddd
[66, 66, 55, 48, 63, 66, 6] 2
190 dddddddddddd
[66, 66, 55, 48, 63, 66, 6] 2
190 dddddddddddd
[87, 17, 9, 93, 74, 74, 55] 1
28
200 dddddddddddd
[13, 9, 70, 84, 96, 73, 12] 6
190 dddddddddddd
[101, 68, 70, 44, 48, 34, 48] 5
190 dddddddddddd
[62, 68, 9, 44, 48, 14, 48] 3
190 dddddddddddd
[62, 68, 9, 44, 48, 14, 48] 3
190 dddddddddddd
[16, 32, 9, 83, 83, 9, 100] 2
190 dddddddddddd
[2, 9, 9, 32, 9, 14, 48] 1
190 dddddddddddd
[2, 9, 9, 32, 9, 14, 48] 1
29
200 dddddddddddd
[33, 1, 35, 27, 40, 32, 18] 8
190 dddddddddddd
[84, 17, 6, 33, 18, 73, 61] 6
190 dddddddddddd
[33, 1, 6, 33, 18, 73, 51] 5
190 dddddddddddd
[33, 1, 6, 33, 18, 73, 51] 5
190 dddddddddddd
[38, 68, 54, 20, 93, 84, 51] 3
190 dddddddddddd
[38, 68, 54, 20, 93, 84, 51] 3
190 dddd

KeyboardInterrupt: 

In [ ]:
timetable

,lh0,lh1,lh2,lh3,lh4,lh5,lh6,lh7,lh8,lh9
0,"[15, 87, 57, 84, 98, 94, 63]","[50, 26, 46, 98, 40, 78, 43]","[13, 93, 76, 62, 42, 69, 39]","[80, 96, 22, 19, 9, 64, 16]","[63, 45, 44, 77, 33, 77, 5]","[90, 85, 29, 53, 17, 17, 69]","[5, 53, 65, 55, 91, 99, 84]","[36, 72, 13, 73, 56, 47, 71]","[23, 76, 45, 22, 70, 42, 48]","[49, 80, 24, 68, 64, 87, 52]"
1,"[72, 40, 34, 2, 4, 1, 11]","[89, 16, 49, 48, 52, 48, 47]","[96, 28, 97, 66, 30, 11, 18]","[71, 83, 20, 60, 25, 21, 68]","[62, 50, 74, 86, 93, 90, 25]","[35, 65, 51, 99, 19, 34, 99]","[31, 2, 0, 70, 18, 74, 66]","[79, 36, 67, 27, 55, 67, 31]","[8, 10, 39, 9, 75, 43, 24]","[73, 95, 6, 88, 12, 33, 89]"
2,"[0, 58, 82, 44, 51, 57, 4]","[83, 54, 88, 21, 91, 15, 75]","[59, 8, 58, 32, 26, 38, 58]","[61, 61, 97, 12, 23, 14, 82]","[6, 94, 10, 28, 28, 1, 28]","[27, 7, 81, 54, 86, 54, 54]","[79, 35, 85, 14, 7, 7, 81]","[101, 101, 41, 3, 41, 56, 41]",Row3_Col8,Row3_Col9
3,Row4_Col0,Row4_Col1,Row4_Col2,Row4_Col3,Row4_Col4,Row4_Col5,Row4_Col6,Row4_Col7,Row4_Col8,Row4_Col9
4,Row5_Col0,Row5_Col1,Row5_Col2,Row5_Col3,Row5_Col4,Row5_Col5,Row5_Col6,Row5_Col7,Row5_Col8,Row5_Col9


In [ ]:
course_metadata.iloc[100]

Course ID                      CS375
Section                            C
Teacher ID      Dr. Fahad Bin Muslim
Credit Hours                       1
Semester                           6
Name: 100, dtype: object

In [ ]:
from collections import Counter

# Example list
my_list = ['apple', 'banana', 'apple', 'orange', 'banana', 'apple']

# Count occurrences
occurrences = Counter(my_list)

# Print the count for each element
print(occurrences['apple'])


3


In [ ]:
# Python3 program to create target string, starting from 
# random string using Genetic Algorithm 

import random 

# Number of individuals in each generation 
POPULATION_SIZE = 100

# Valid genes 
GENES = '''abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOP 
QRSTUVWXYZ 1234567890, .-;:_!"#%&/()=?@${[]}'''

# Target string to be generated 
TARGET = "I love GeeksforGeeks"

class Individual(object): 
	''' 
	Class representing individual in population 
	'''
	def __init__(self, chromosome): 
		self.chromosome = chromosome 
		self.fitness = self.cal_fitness() 

	@classmethod
	def mutated_genes(self): 
		''' 
		create random genes for mutation 
		'''
		global GENES 
		gene = random.choice(GENES) 
		return gene 

	@classmethod
	def create_gnome(self): 
		''' 
		create chromosome or string of genes 
		'''
		global TARGET 
		gnome_len = len(TARGET) 
		return [self.mutated_genes() for _ in range(gnome_len)] 

	def mate(self, par2): 
		''' 
		Perform mating and produce new offspring 
		'''

		# chromosome for offspring 
		child_chromosome = [] 
		for gp1, gp2 in zip(self.chromosome, par2.chromosome):	 

			# random probability 
			prob = random.random() 

			# if prob is less than 0.45, insert gene 
			# from parent 1 
			if prob < 0.45: 
				child_chromosome.append(gp1) 

			# if prob is between 0.45 and 0.90, insert 
			# gene from parent 2 
			elif prob < 0.90: 
				child_chromosome.append(gp2) 

			# otherwise insert random gene(mutate), 
			# for maintaining diversity 
			else: 
				child_chromosome.append(self.mutated_genes()) 

		# create new Individual(offspring) using 
		# generated chromosome for offspring 
		return Individual(child_chromosome) 

	def cal_fitness(self): 
		''' 
		Calculate fitness score, it is the number of 
		characters in string which differ from target 
		string. 
		'''
		global TARGET 
		fitness = 0
		for gs, gt in zip(self.chromosome, TARGET): 
			if gs != gt: fitness+= 1
		return fitness 

# Driver code 
def main(): 
	global POPULATION_SIZE 

	#current generation 
	generation = 1

	found = False
	population = [] 

	# create initial population 
	for _ in range(POPULATION_SIZE): 
				gnome = Individual.create_gnome() 
				population.append(Individual(gnome)) 

	while not found: 

		# sort the population in increasing order of fitness score 
		population = sorted(population, key = lambda x:x.fitness) 

		# if the individual having lowest fitness score ie. 
		# 0 then we know that we have reached to the target 
		# and break the loop 
		if population[0].fitness <= 0: 
			found = True
			break

		# Otherwise generate new offsprings for new generation 
		new_generation = [] 

		# Perform Elitism, that mean 10% of fittest population 
		# goes to the next generation 
		s = int((10*POPULATION_SIZE)/100) 
		new_generation.extend(population[:s]) 

		# From 50% of fittest population, Individuals 
		# will mate to produce offspring 
		s = int((90*POPULATION_SIZE)/100) 
		for _ in range(s): 
			parent1 = random.choice(population[:50]) 
			parent2 = random.choice(population[:50]) 
			child = parent1.mate(parent2) 
			new_generation.append(child) 

		population = new_generation 

		print("Generation: {}\tString: {}\tFitness: {}".format(generation, "".join(population[0].chromosome), population[0].fitness))

		generation += 1

	
	print("Generation: {}\tString: {}\tFitness: {}".format(generation, "".join(population[0].chromosome), population[0].fitness))
 

if __name__ == '__main__': 
	main() 


)
2


d
2
h
A
"
L


?
n
y
}
R
O
8
9
h
A
e
 
x
R
r
,
{
J
j
$
H
:
U
[
J
t
.
/
Y
;
$
k
d
$
:
h
z
,
V


S
e
O
S
[
$
x
i
9
X
q
[
(
B
D
b
P
[
8
L
H
,
 
}
$
O
_
L
H
Z
#
7
N
:
Q
V
g
v
S
7
9
-
5
(
1
k
H
b
Y
9
5
A
R
d
 
L
X
o
/
d
q
 
[
J
[
P
n
t
$
R
G
6
[
N
&
K
c
1
o
E
 
:
(
;
O
$
B
 
3
f
U
B
T
@
?
u
n
O
s
7
-
 
#
_
P
7
t
B
]
t
2
R
t
w
O
:
#
R
_
 
W
S
F
p
 
:
!
v
)
h
I
j
6
4
q
"
Y


J
"
Q
Q
 
s
x
 
X
9
P
 
%
3
;
w
(
s
G
f
/
L
!
u
{
d
R
Y
(
 
K
@
c
/
d
;
T
Z
D
z
g
I
R
 
_
b
Y
7
l
#
f
!
 
S
D
R
:
[
&
e
T
G
X
R
M
e
#
7
Q
8
 
U
h
#
]
{
2
S
w
G
-
=
v
p
N
/
b
6
9
R
y
f
.
d
b
z
b
D
k
6
I
[
Z
@
,
e
Y
L
 
:
]
F
S
8
k
T
g
h
-
 
I
P
.
"
M
@
,
&
M
4
C
 
 
x
%
b
M
o
e
@
:
i
9
,
%
/
(
p
J
:
g
e
a
D
?
4
u
F


S
2
 
L
/
O
G
;
6
-
R
:
J
j
Y
o
o
9
}
q
y
p
h
9
4
a
 
&
8
1
q
@
u
q
T
k
f
x
7
q
Q
 
b
Q
/
&
G
H
:
v
1


E
&
B
H
f
1
d
_
 
o
d
l
[
Q
D
s
)
2
%
W
#
s
=
R
.
;
#
m
X
Z
[
Y
P
G
Y
R
%
4
K
z
O
i
J
 
O
z
1


r
z
]
9
@
b
g
d
N
A
k
A
{
P
P
j
g
w
I
!
]
D
 
P
Z
M
O
y
v
V
:
g
h
2
}
L
V
q
z
;
/
;
!
[
T
5
V
x
[
 
u
p
B
